# Script to generate Synthetic Data from Telemetry Data

In [ ]:
from Framework.DataGeneration.Simulator import FlightSimulation
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# get project path and set data directory for notebook
PROJECT_DIR = os.getcwd()

Establish the different locations for the radar and level of noise in the measurements to create diverse and heterogeneous data from each trajectory intentionclass.

In [ ]:
# Create a flight simulator to model different mission profiles under different radar and sensor noise levels
Simulation = FlightSimulation() # Refer to the DataGeneration files
noise_levels = [1.0, 3.0]
#noise_levels = [1.0, 2.0, 3.0]
#bearing_rads = [0.6, 0.8]
radar_posns = [[0,0], [250, 250], [500, 0], [0, 500], [150, 50], [50, 150], [750, 750]]

SIM_SETTINGS = {'mapping_flight' : {'VEL_COEF' : 0.1, 
                                       'REPEAT_NUMBER' : 2,
                                       'noise_levels' : [1.0, 3.0],
                                       'radar_posns' : [[0,0], [250, 250], 
                                                        [500, 0], [0, 500], 
                                                        [150, 50], [50, 150], 
                                                        [750, 750]]},
 
                   'perimeter_flight' : {'VEL_COEF' : 0.1, 
                                         'REPEAT_NUMBER' : 2,
                                         'noise_levels' : [1.0, 3.0],
                                         'radar_posns' : [[0,0], [250, 250], 
                                                          [500, 0], [0, 500], 
                                                          [150, 50], [50, 150], 
                                                          [750, 750]]},
 
                   'package_delivery' : {'VEL_COEF' : 0.1, 
                                         'REPEAT_NUMBER' : 1,
                                         'noise_levels' : [1.0, 3.0],
                                         'radar_posns' : [[0,0], [250, 250], 
                                                          [500, 0], [0, 500], 
                                                          [150, 50], [50, 150], 
                                                          [750, 750]]},
 
                   'point_point_flight' : {'VEL_COEF' : 0.1, 
                                           'REPEAT_NUMBER' : 2,
                                           'noise_levels' : [1.0, 3.0],
                                           'radar_posns' : [[0,0], [250, 250], 
                                                            [500, 0], [0, 500], 
                                                            [150, 50], [50, 150], 
                                                            [750, 750]]},
                   
                  'other' : {'VEL_COEF' : 0.1, 
                                          'REPEAT_NUMBER' : 1,
                                           'noise_levels' : [1.0, 3.0],
                                           'radar_posns' : [[0,0], [250, 250], 
                                                            [500, 0], [0, 500], 
                                                            [150, 50], [50, 150], 
                                                            [750, 750]]}}

#list of intent classes to generate data for
INTENTS = ['mapping_flight',
           'perimeter_flight',
           'package_delivery',
           'point_point_flight',
           'other']

Create a new folder to store the new simulated radar tracks. This process can be applied for $m$ trajectory intention classes to increase the richness of the data

In [ ]:
# iterate through each intent and simulate data accordingly
for intent_name in INTENTS:
    
    # set appropriate source and destination directories
    DATA_DIR = os.path.join(PROJECT_DIR, f'ResearchData/{intent_name}')
    DEST_FOLDER = os.path.join(PROJECT_DIR, f'ResearchData/simulated_measurements/{intent_name}')
    isExist = os.path.exists(DEST_FOLDER)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(DEST_FOLDER)
        print("The new directory is created!")

    FLIGHT_FILES = [x for x in os.listdir(DATA_DIR) if x.endswith('csv')]
    print(f"{75*'-'}")
    print(f"Starting simulation for intent class: {intent_name}...")
    print(f"Number of files to base simulations on: {len(FLIGHT_FILES)}")
    
    saved_names, original_files = [], []
    uav_types = []
    noises = []
    radar_xs, radar_ys = [], []
    sampling_times = []

    # iterate through all unique flights and generate simulations
    for filename in tqdm(FLIGHT_FILES):
        
        # load each main file once, and obtain uav type & label
        flight_df = pd.read_csv(os.path.join(DATA_DIR, filename))
        uav_type = flight_df['uav_category'][0]
    
        # get filename without ext
        filename_stub = filename.split('.')[0]
    
        # keep counter for naming simulated flights
        i = 0
    
        # iterate through all possible combinations of settings, generating measurements
        # each time
        for combo in itertools.product(SIM_SETTINGS[intent_name]['noise_levels'], 
                                       SIM_SETTINGS[intent_name]['radar_posns']):

            # repeat each measurement process N times
            for _ in range(SIM_SETTINGS[intent_name]['REPEAT_NUMBER']):

                # get simulated measurements for flight and settings config
                temp_df = Simulation.get_simulated_flight_df(
                            filename, DATA_DIR, 
                            noise_factor=combo[0], 
                            velocity_coef=SIM_SETTINGS[intent_name]['VEL_COEF'],
                            offset_x=combo[1][0], 
                            offset_y=combo[1][1])

                # add additional columns to data before saving
                temp_df['uav_type'] = uav_type

                # set appropriate filename, save as .csv to dest
                save_name = f"{filename_stub}_{i + 1}.csv"
                temp_df.to_csv(os.path.join(DEST_FOLDER, save_name), index=False)
                #print(f"Saved data file: {save_name} to {DEST_FOLDER}")

                # determine sampling time
                sampling_time = temp_df['time'].diff().mean().total_seconds()

                # append meta details to form a meta-data dataframe later
                saved_names.append(save_name)
                original_files.append(filename)
                uav_types.append(uav_type)
                noises.append(float(combo[0]))
                radar_xs.append(combo[1][0])
                radar_ys.append(combo[1][1])
                sampling_times.append(sampling_time)

                # increment i (end of current loop)
                i += 1

    # convert into numpy array
    saved_names = np.array(saved_names)
    original_files = np.array(original_files)
    uav_types = np.array(uav_types)
    noises = np.array(noises)
    radar_xs = np.array(radar_xs)
    radar_ys = np.array(radar_ys)
    sampling_times = np.array(sampling_times)

    # convert all results into dataframe format
    meta_data_df = pd.DataFrame({'filename' : saved_names,
                                 'original_file' : original_files,
                                 'uav_type' : uav_types,
                                 'noise_level' : noises,
                                 'radar_x' : radar_xs,
                                 'radar_y' : radar_ys,
                                 'sampling_time' : sampling_times})

    # obtain current date and set save file name for final exported Excel File
    CURRENT_DATE = datetime.today().strftime('%Y%m%d')

    # save dataframe with meta-data into higher level directory
    meta_data_df.to_csv(os.path.join(PROJECT_DIR, 'ResearchData', 
                        f"{CURRENT_DATE}-{intent_name}_data_info.csv"),
                       index=False)
    
    print(f"Saved {len(os.listdir(DEST_FOLDER))} simulations for intent: {intent_name}.\n")